In [2]:
import json
import pandas as pd
import os
from enum import IntEnum

In [4]:
PATH1 = "../../data/pilot1/game_data/game_logs/"
PATH2 = "../../data/pilot2/game_data/game_logs/"
# P60d520539b147c4acaf44b77_voff_m1_g1_t2021_7_10_1_4

In [3]:
with open(PATH1 + 'P5ee6d2494fd04045b988a12d_voff_m1_g1_t2021_7_10_0_47.json', 'r') as f:
    data = json.loads(f.read())
df = pd.json_normalize(data, record_path = ['frames'])
df['player_id'] = data['player_id']
df['mode'] = data['mode']

In [4]:
# df.head()
# df.info()

In [5]:
# key events to log

ai_deaths = []
ai_shots = []
right_to_left = []
left_to_right = []

up_signals = []
down_signals = []

ai_side_over = []
player_side_over = []
all_over = []

shots = []

final_frame = None

In [6]:
ai_deaths = []
for event in data['events']:
    if event['killed'] == 'AI':
        ai_deaths.append(event['frame'])
    if event['killer'] == 'AI' and event['type'] == 'SHOT':
        ai_shots.append(event['frame'])

In [7]:
# chose to ignore side changes that happened near AI_agent deaths; not sure if this is the best idea

last_ai_x = 1000
last_lives = 4
last_left_enemies_count = 30
last_right_enemies_count = 30
recent_death = False

right_to_left = []
left_to_right = []


for frame in data['frames']:
    up = frame['signal_up']
    down = frame['signal_down']
    ai_x = frame['ai_position']
    frame_num = frame['frame_number']
    left_count = len(frame['enemies_left_positions'])
    right_count = len(frame['enemies_right_positions'])
    shoot = frame['ai_actual_action']['shoot']
    
    if last_left_enemies_count > 0 and left_count == 0:
        player_side_over.append(frame_num)
    if last_right_enemies_count > 0 and right_count == 0:
        ai_side_over.append(frame_num)
    if last_left_enemies_count + last_right_enemies_count > 0 and left_count == 0 and right_count == 0:
        all_over.append(frame_num)
    
    if shoot:
        shots.append(frame_num)
        
    if up:
        up_signals.append(frame_num)
    if down:
        down_signals.append(frame_num)
        
    for n in ai_deaths:
        if n < frame_num and n > frame_num - 50:
            recent_death = True
    if not recent_death:
        if last_ai_x >= 400 and ai_x < 400:
            right_to_left.append(frame_num)
        if last_ai_x <= 400 and ai_x > 400:
            left_to_right.append(frame_num)
    last_ai_x = ai_x
    last_left_enemies_count = left_count
    last_right_enemies_count = right_count
    
final_frame = data['frames'][-1]['frame_number']
        

In [8]:
print(f'right to left: {right_to_left}')
print(f'left to right: {left_to_right}')
print(f'up signals: {up_signals}')
print(f'down signals: {down_signals}')
print(f'ai_deaths: {ai_deaths}')
print(f'ai_shots: {ai_shots}')
print(f'player side over: {player_side_over}')
print(f'ai side over: {ai_side_over}')

right to left: [813, 2050]
left to right: [1209, 2988]
up signals: [165, 480, 708, 1145, 1621, 2530, 2922, 3115]
down signals: []
ai_deaths: []
ai_shots: [48, 90, 128, 166, 208, 303, 340, 421, 854, 921, 958, 993, 1030, 1113, 1267, 1304, 1339, 1375, 1411, 1488, 1584, 1624, 1696, 1736, 1818, 1862, 2227, 2272, 2319, 2453, 2625, 2739, 2783, 3054, 3115]
player side over: [2790]
ai side over: [3121]


In [9]:
type(data['frames'][0]['frame_number'])

int

In [10]:
final_frame

3121

In [11]:
def search_recent(l, frame_num, l_search_range=50, right=False, r_search_range=50, left=True):
    for x in l:
        if left and x <= frame_num and x > frame_num - l_search_range:
            return x
        if right and x >= frame_num and x < frame_num + r_search_range:
            return x
    return False

def search_recent_df(df, col, frame_num, search_range=50, right=False, left=True):
    d = None
    if left and right:
        d = df.loc[(df['frame_number'] > frame_num - search_range) & (df['frame_number'] < frame_num + search_range)]
    elif left:
        d = df.loc[(df['frame_number'] > frame_num - search_range) & (df['frame_number'] < frame_num)]
    elif right:
        d = df.loc[(df['frame_number'] < frame_num + search_range) & (df['frame_number'] > frame_num)]
    if len(d.loc[d[col] == True]) > 0:
        return True
    return False
            

In [12]:
df['enemies_right_low'] = [z > 440 for z in [max([x[1] for x in y]) if len(y) > 0 else 0 for y in df['enemies_right_positions']]]
df['enemies_right_low_2'] = [z > 400 for z in [max([x[1] for x in y]) if len(y) > 0 else 0 for y in df['enemies_right_positions']]]

In [13]:
class Code(IntEnum):
    SHOT = 1
    RIGHT_TO_LEFT = 2
    LEFT_TO_RIGHT = 3
    AI_DEATH = 4
    INACTIVE = 5 #not shooting
    REACTIVE = 6
    AI_FINISH = 7
    P_FINISH = 8
    FINISH = 9
    TESTING = 10
    MISTAKE = 11
    R_ENEMIES_LOW = 12
    IGNORANT = 13 # on left side but right is low
    

In [14]:
def down_cat(frame_num, search_range):
    codes = []
    if search_recent(right_to_left, frame_num, search_range, True, 15):
        codes.append(Code.RIGHT_TO_LEFT)
    if search_recent(ai_deaths, frame_num, search_range):
        codes.append(Code.AI_DEATH)
    if not search_recent(shots, frame_num, search_range):
        codes.append(Code.INACTIVE)
    if search_recent_df(df, 'enemies_right_low', frame_num, search_range):
        codes.append(Code.R_ENEMIES_LOW)
    if (df.at[frame_num,'ai_position'] < 400) and (df.at[frame_num,'player_position'] < 400) and (df.at[frame_num, 'enemies_right_low_2'] == True):
        codes.append(Code.IGNORANT)
    return codes

In [15]:
def up_cat(frame_num, search_range):
    codes = []
    if search_recent(right_to_left, frame_num, search_range, True, 15):
        codes.append(Code.RIGHT_TO_LEFT)
    if search_recent(ai_shots, frame_num, search_range):
        codes.append(Code.SHOT)
    x = search_recent(shots, frame_num, search_range)
    if x and not search_recent(shots, x, search_range):
        codes.append(Code.REACTAIVE)
    
    a_o = search_recent(ai_side_over, frame_num, search_range, True, 30)
    p_o = search_recent(player_side_over, frame_num, search_range, True, 30)
    if a_o and df.at[a_o, 'ai_position'] > 400:
        codes.append(Code.AI_FINISH)
    if p_o and df.at[p_o, 'ai_position'] < 400:
        codes.append(Code.P_FINISH)
    if search_recent(all_over, frame_num, search_range, True, 30):
        codes.append(Code.FINISH)
        
    return codes

In [6]:
dfs = []
for file in sorted(os.listdir(PATH1)):
    if not file.startswith('.'):
        with open(PATH1 + file, 'r') as f:
            data = json.loads(f.read())
        df = pd.json_normalize(data, record_path = ['frames'])
        df['player_id'] = data['player_id']
        df['mode'] = data['mode']
        dfs.append(df[['player_id', 'mode', 'signal_up', 'signal_down', 'tried_signal_up', 'tried_signal_down', 'frame_number']][df['signal_up'] | df['signal_down']])
        print('loading: {} mode{}'.format(data['player_id'], data['mode']))

loading: 5ee6d2494fd04045b988a12d mode1
loading: 5ee6d2494fd04045b988a12d mode2
loading: 60d520539b147c4acaf44b77 mode1
loading: 60d520539b147c4acaf44b77 mode2
loading: 60e5e62659dc1912bc43ed35 mode1
loading: 60e5e62659dc1912bc43ed35 mode2
loading: 60e7a66b2cb32cc90fade4e6 mode1
loading: 60e7a66b2cb32cc90fade4e6 mode2


In [16]:
for signal_frame in down_signals:
    print(f'{signal_frame}: {down_cat(signal_frame, 150)}')

In [17]:
for signal_frame in up_signals:
    print(f'{signal_frame}: {up_cat(signal_frame, 150)}')

165: [<Code.SHOT: 1>]
480: [<Code.SHOT: 1>]
708: []
1145: [<Code.SHOT: 1>]
1621: [<Code.SHOT: 1>]
2530: [<Code.SHOT: 1>]
2922: [<Code.SHOT: 1>, <Code.P_FINISH: 8>]
3115: [<Code.SHOT: 1>, <Code.AI_FINISH: 7>, <Code.FINISH: 9>]
